In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import warnings



np.random.seed(42)
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy'

#### Importera delar från del1:

In [ ]:
def visa_statistik(df):
    kolumner = ["age", "systolic_bp", "cholesterol", "weight" ]
    for kol in kolumner:
        print(f"{kol}: medel={df[kol].mean():.1f}, median={df[kol].median():.1f}")

kolumner = ["age", "weight", "height", "systolic_bp", "cholesterol"]
stats_df = pd.DataFrame(columns=["mean", "median", "min", "max"], index=kolumner)

for col in kolumner:
    col_vals = df[col].dropna()
    stats_df.loc[col] = [np.mean(col_vals), np.median(col_vals), np.min(col_vals), np.max(col_vals)]

def skapa_grafer(df):
    fig, axs = plt.Subplots(2, 2, figsize=(12, 8))

    num_kol = ["age", "weight", "systolic_bp", "cholesterol" ]
    sns.heatmap(df[num_kol].corr(),annot=True, ax=axs[0,0])

    axs[0,1].scatter(df["age"], df["systolic_bp"], alpha=0.5)
    axs[0,1].set_title("Ålder vs Blodtryck")
    axs[0,1].boxplot([df[df["sex"]=="M"]["systolick_bp"],
                    df[df["sex"]=="F"]["systolic_bp"]],
                    labels=["Män", "Kvinnor"])
    
    sjukdom_per_kön = df.groupby("sex")["disease"].mean() * 100
    axs[1,1].bar("sjuk_per_kön.index, sjuk_per_kön.values")
    
    plt.tight_layout()
    plt.show()
    
df = pd.read_csv("health_study_dataset.csv")
print(df.head())
df.info()

NameError: name 'df' is not defined

#### Andel personer med sjukdom

In [ ]:
sjuckdom_grupp = df["disease"].mean()
print(f"Andel med sjukdom:{sjuckdom_grupp:3f}")

np .random.seed(42)
simulerad = np.random.binomial(n=1, p=sjuckdom_grupp, size=1000)
andel_simulerad =np.mean(simulerad)
print(f"andel med sjukdom i simulerad data: {andel_simulerad:3f}")

differens = sjuckdom_grupp - andel_simulerad
print(f"Skillnaden i andel med sjukdom mellan verklig och simulerad data: {differens:3f}")

#### Klasser:

In [ ]:
class HealsAnalyz:

    def __init__(self, data):
        self.data = data

    def sammanfattning(self):
        return{
            "antal": len(self.data),
            "antal_sjuk": self.data["disease"].mean() * 100,
            "medelålder": self.data["age"].mean()
        }
    
    def regression(self, x_kol, y_kol):
        x = self.data[[x_kol]] if isinstance(x_kol, str) else self.data[x_kol]
        y =self.data[y_kol]

        
        model = LinearRegression()
        model.fit(x, y)

        return {
            "r2": model.score(x, y),
            "koeff": model.coef_,
            "intercept": model.intercept_
        }
   
analysator = HealsAnalyz(df)
print("sammanfattning:")
for nyckel, värde in analysator.sammanfattning().items():
        print(f"{nyckel}: {värde}")

#### Linjer algebra - Regression:

In [ ]:
print("\nREGRESSION ANALYS:")

reg1 = analysator.regression('age', 'systolic_bp')
print(f"Ålder -> Blodtryck: R² = {reg1['r2']:.3f}")

reg2 = analysator.regression(['age', 'weight'], 'systolic_bp')
print(f"Ålder + Vikt -> Blodtryck: R² = {reg2['r2']:.3f}")

#### Linjer Algebra - PCA:

In [ ]:
print("\nPCA ANALYS:")
from sklearn.decomposition import PCA

num_data = df[['age', 'systolic_bp', 'cholesterol', 'weight']].dropna()

pca =PCA(n_components=2)
pca_result = pca.fit_transform(num_data)

print(f"PC1 förlkarar {pca.explained_variance_ratio_[0]*100:.0f}% av varjansen")
print(f"PC2 förlkarar {pca.explained_variance_ratio_[1]*100:.1f}% av varjansen")

plt.figure(figsize=(7, 5))

plt.scatter(pca_result[:,0], pca_result[:,1], alpha=0.5)
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.title('PCA: PCA1 vs PCA2')
plt.show()



#### Utökad Analys:

In [ ]:
print("\nUTÖKAD ANALYS:")
df['åldersgräns'] =pd.cut(df['age'], bins=[0, 40, 60, 100], labels=['<40', '40-60', '60+'])

sjuk_per_ålder = df.groupby('åldersgräns')['disease'].mean() * 100
print("sjukdomrisk per ålder:")

for ålder, risk in sjuk_per_ålder.items():
    print(f" {ålder}: {risk:.1f}%")

    bp_män = df[df['sex']=='M']['systolic_bp'].mean()
    bp_kvinnor = df[df['sex']=='F']['systolic_bp'].mean()
    print(f'\nBlodtryck: män={bp_män:.1f}, kvinnor={bp_kvinnor:.1f}')

    fig,(ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))


    ax1.bar(sjuk_per_ålder.index.astype(str), sjuk_per_ålder.values)
    ax1.set_title('sjukdomsrisk per ålder')

    ax2.boxplot([df[df['disease']==0]['weight'], df[df['disease']==1]['weight']], labels= ['Frisk', 'Sjuk'])
    ax2.set_title('Vikt för sjuka och friska')

    plt.tight_layout()
    plt.show()
    print(df.åldersgräns)